In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
import calendar
import pickle
import joblib
import pyodbc

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, auc
from imblearn.under_sampling import TomekLinks

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)

In [2]:
model = joblib.load('xgb_best_model_task1.pkl')
model_kmeans = joblib.load('kmeans_model_new.pkl')
vectorizer = joblib.load('vectorizer_task1.pkl')

In [3]:
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.04994580917280129, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05122322664789635,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=30, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1903, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [4]:
details = ('DRIVER={SQL SERVER};'
           'SERVER=DESKTOP-OEN5051\SQLEXPRESS;'
           'DATABASE=Bank Loan;'
           'Trusted_Connection= yes;')

In [5]:
connection = pyodbc.connect(details)
cursor = connection.cursor()

In [6]:
sql_query = "SELECT * FROM [Bank Loan].[dbo].[financial_loan]"

In [7]:
df_all = pd.read_sql_query(sql_query, connection)
df_all.shape

(38576, 24)

In [8]:
# df_all = pd.read_csv("data.csv")
# df_all = df_all.dropna()

In [9]:
df_all

,id,address_state,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,purpose,sub_grade,term,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,Term_Mnths,Profit,Profit_Percent
0,1077430,GA,< 1 year,RYDER,C,Rent,2021-02-11,2021-09-13,2021-04-13,Charged Off,2021-05-13,Car,C4,60 months,30000.0,0.0100,59.830002,0.1527,2500.0,4,1009,60.0,-1491.0,-59.640000
1,1072053,CA,9 years,MKC ACCOUNTING,E,Rent,2021-01-01,2021-12-14,2021-01-15,Fully Paid,2021-02-15,Car,E1,36 months,48000.0,0.0535,109.430000,0.1864,3000.0,4,3939,36.0,939.0,31.300000
2,1069243,CA,4 years,CHEMAT TECHNOLOGY INC,C,Rent,2021-01-05,2021-12-12,2021-01-09,Charged Off,2021-02-09,Car,C5,36 months,50000.0,0.2088,421.649994,0.1596,12000.0,11,3522,36.0,-8478.0,-70.650000
3,1041756,TX,< 1 year,BARNES DISTRIBUTION,B,Mortgage,2021-02-25,2021-12-12,2021-03-12,Fully Paid,2021-04-12,Car,B2,60 months,42000.0,0.0540,97.059998,0.1065,4500.0,9,4911,60.0,411.0,9.133333
4,1068350,IL,10 > years,J&J STEEL INC,A,Mortgage,2021-01-01,2021-12-14,2021-01-15,Fully Paid,2021-02-15,Car,A1,36 months,83000.0,0.0231,106.529999,0.0603,3500.0,28,3835,36.0,335.0,9.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38571,803452,NJ,< 1 year,JOSEPH M SANZARI COMPANY,C,Mortgage,2021-07-11,2021-05-16,2021-05-16,Current,2021-06-16,Wedding,C1,60 months,100000.0,0.1986,551.640015,0.1299,24250.0,33,31946,60.0,7696.0,31.736082
38572,970377,NY,8 years,SWAT FAME,C,Rent,2021-10-11,2021-04-16,2021-05-16,Current,2021-06-16,Wedding,C1,60 months,50000.0,0.0458,579.719971,0.1349,25200.0,18,31870,60.0,6670.0,26.468254
38573,875376,CA,5 years,ANAHEIM REGIONAL MEDICAL CENTER,D,Rent,2021-09-11,2021-05-16,2021-05-16,Current,2021-06-16,Wedding,D5,60 months,65000.0,0.1734,627.929993,0.1749,25000.0,20,35721,60.0,10721.0,42.884000
38574,972997,NY,5 years,BROOKLYN RADIOLOGY,D,Rent,2021-10-11,2021-05-16,2021-05-16,Current,2021-06-16,Wedding,D5,60 months,368000.0,0.0009,612.719971,0.1825,24000.0,9,33677,60.0,9677.0,40.320833


In [10]:
df_all['total_payment'].max()

58564

# Pre-Processing

In [11]:
df_all['emp_title'].nunique()

27120

In [12]:
df_all['emp_title'] = df_all['emp_title'].fillna('None')
emp_title = df_all['emp_title']
emp_title_transform = vectorizer.transform(emp_title.values)
predictions = model_kmeans.predict(emp_title_transform)
df_all['Cluster'] = predictions

In [13]:
cluster_label = {0: 'Financial Services',
    1: 'None', 2:'Public Service', 3: 'Public Service', 4:'Healthcare', 5: 'Financial Services', 6:'Healthcare', 7: 'Military', 
    8:'Insurance Services', 9: 'Public Service', 10:'Financial Services', 11: 'Organizations', 12:'Financial Services', 
    13: 'Corporations', 14:'Healthcare', 15: 'Self-Employed', 16:'Organizations', 17: 'Education', 18:'Facilities', 
    19: 'Organizations', 20:'Business', 21: 'Business', 22:'Service Providers', 23: 'Military', 24:'Public Service',
    25:'Public Service',26:'Healthcare',27:'Retail'
}

df_all['job_title'] = df_all['Cluster'].map(cluster_label)

In [14]:
state_map = {
    'CA': 'California', 'NY': 'New York', 'TX': 'Texas', 'CT': 'Connecticut', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'KY': 'Kentucky', 'MS': 'Mississippi', 'KS': 'Kansas',
    'GA': 'Georgia', 'NC': 'North Carolina', 'NJ': 'New Jersey', 'WA': 'Washington', 'OH': 'Ohio',
    'AZ': 'Arizona', 'IL': 'Illinois', 'CO': 'Colorado', 'WI': 'Wisconsin', 'FL': 'Florida',
    'IN': 'Indiana', 'VA': 'Virginia', 'ME': 'Maine', 'NE': 'Nebraska', 'NM': 'New Mexico',
    'MO': 'Missouri', 'UT': 'Utah', 'LA': 'Louisiana', 'SC': 'South Carolina', 'AL': 'Alabama',
    'TN': 'Tennessee', 'NH': 'New Hampshire', 'SD': 'South Dakota', 'DE': 'Delaware', 'WY': 'Wyoming',
    'OR': 'Oregon', 'MN': 'Minnesota', 'DC': 'District of Columbia', 'HI': 'Hawaii', 'NV': 'Nevada',
    'VT': 'Vermont', 'IA': 'Iowa', 'ID': 'Idaho', 'RI': 'Rhode Island', 'PA': 'Pennsylvania',
    'AR': 'Arkansas', 'OK': 'Oklahoma', 'AK': 'Alaska', 'MT': 'Montana', 'WV': 'West Virginia'
}

df_all['address_state_full'] = df_all.apply(lambda row: state_map.get(row['address_state']), axis=1)

In [15]:
def mapping(x):
    if x == '10 > years':
        return '11 years'
    if x == '< 1 year':
        return '0 year'
    else:
        return x
    
df_all['emp_length'] = df_all['emp_length'].apply(mapping)

df_all['issue_date'] = pd.to_datetime(df_all['issue_date'])
df_all['issue_date_month'] = df_all['issue_date'].dt.month.apply(lambda x: calendar.month_name[int(x)])

In [16]:
df_all['loan_statuss'] = df_all['loan_status'].apply(lambda x: 'Paid' if x in ['Current', 'Fully Paid'] else 'Charged Off')

le = LabelEncoder()
df_all['label'] = le.fit_transform(df_all['loan_statuss'])

In [17]:
features = ['annual_income','dti','installment','int_rate','loan_amount','total_acc','total_payment', 'emp_length', 'term', 'grade', 'purpose',  'job_title', 'issue_date_month' ,  'address_state_full',  'sub_grade', 'home_ownership']
X = df_all[features]
y = df_all['label']

In [18]:
print(X.shape, y.shape)

(38576, 16) (38576,)


In [19]:
X.head()

,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length,term,grade,purpose,job_title,issue_date_month,address_state_full,sub_grade,home_ownership
0,30000.0,0.0100,59.830002,0.1527,2500.0,4,1009,0 year,60 months,C,Car,Organizations,February,Georgia,C4,Rent
1,48000.0,0.0535,109.430000,0.1864,3000.0,4,3939,9 years,36 months,E,Car,Organizations,January,California,E1,Rent
2,50000.0,0.2088,421.649994,0.1596,12000.0,11,3522,4 years,36 months,C,Car,Organizations,January,California,C5,Rent
3,42000.0,0.0540,97.059998,0.1065,4500.0,9,4911,0 year,60 months,B,Car,Organizations,February,Texas,B2,Mortgage
4,83000.0,0.0231,106.529999,0.0603,3500.0,28,3835,11 years,36 months,A,Car,Organizations,January,Illinois,A1,Mortgage


In [20]:
#X.to_csv('new_df.csv', index = False)

In [21]:
X['job_title'].unique()

array(['Organizations', 'Corporations', 'None', 'Insurance Services',
       'Self-Employed', 'Public Service', 'Facilities',
       'Financial Services', 'Retail', 'Healthcare', 'Business',
       'Service Providers', 'Military', 'Education'], dtype=object)

In [22]:
def remove_states(x):
    lst = ['Vermont', 'Tennessee', 'South Dakota', 'Mississippi', 'Oklahoma']
    if x in lst:
        return 'Others'
    else:
        return x

In [23]:
X['address_state_full_update'] = X['address_state_full'].apply(remove_states)

In [24]:
categorical = ['emp_length', 'term', 'grade', 'purpose',  'job_title', 'issue_date_month' ,  'address_state_full_update',  'sub_grade', 'home_ownership']
X = pd.get_dummies(X, columns=categorical, dtype=int)

In [25]:
X

,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,address_state_full,emp_length_0 year,emp_length_1 year,emp_length_11 years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,term_36 months,term_60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,purpose_Car,purpose_Credit card,purpose_Debt consolidation,purpose_Educational,purpose_Home improvement,purpose_House,purpose_Major purchase,purpose_Medical,purpose_Moving,purpose_Other,purpose_Renewable_energy,purpose_Small business,purpose_Vacation,purpose_Wedding,job_title_Business,job_title_Corporations,job_title_Education,job_title_Facilities,job_title_Financial Services,job_title_Healthcare,job_title_Insurance Services,job_title_Military,job_title_None,job_title_Organizations,job_title_Public Service,job_title_Retail,job_title_Self-Employed,job_title_Service Providers,issue_date_month_April,issue_date_month_August,issue_date_month_December,issue_date_month_February,issue_date_month_January,issue_date_month_July,issue_date_month_June,issue_date_month_March,issue_date_month_May,issue_date_month_November,issue_date_month_October,issue_date_month_September,address_state_full_update_Alabama,address_state_full_update_Alaska,address_state_full_update_Arizona,address_state_full_update_Arkansas,address_state_full_update_California,address_state_full_update_Colorado,address_state_full_update_Connecticut,address_state_full_update_Delaware,address_state_full_update_District of Columbia,address_state_full_update_Florida,address_state_full_update_Georgia,address_state_full_update_Hawaii,address_state_full_update_Idaho,address_state_full_update_Illinois,address_state_full_update_Indiana,address_state_full_update_Iowa,address_state_full_update_Kansas,address_state_full_update_Kentucky,address_state_full_update_Louisiana,address_state_full_update_Maine,address_state_full_update_Maryland,address_state_full_update_Massachusetts,address_state_full_update_Michigan,address_state_full_update_Minnesota,address_state_full_update_Missouri,address_state_full_update_Montana,address_state_full_update_Nebraska,address_state_full_update_Nevada,address_state_full_update_New Hampshire,address_state_full_update_New Jersey,address_state_full_update_New Mexico,address_state_full_update_New York,address_state_full_update_North Carolina,address_state_full_update_Ohio,address_state_full_update_Oregon,address_state_full_update_Others,address_state_full_update_Pennsylvania,address_state_full_update_Rhode Island,address_state_full_update_South Carolina,address_state_full_update_Texas,address_state_full_update_Utah,address_state_full_update_Virginia,address_state_full_update_Washington,address_state_full_update_West Virginia,address_state_full_update_Wisconsin,address_state_full_update_Wyoming,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_Mortgage,home_ownership_None,home_ownership_Other,home_ownership_Own,home_ownership_Rent
0,30000.0,0.0100,59.830002,0.1527,2500.0,4,1009,Georgia,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,48000.0,0.0535,109.430000,0.1864,3000.0,4,3939,California,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,

In [26]:
X = X.drop(columns = ['address_state_full'], axis = 1)

In [27]:
X.shape

(38576, 153)

In [28]:
numerical = ['annual_income', 'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc','total_payment']
scaler = StandardScaler()
X_train_numerical = X[numerical]

X_train_scaled = scaler.fit_transform(X_train_numerical)

X[numerical] = X_train_scaled

In [29]:
X = X.drop(columns = ['home_ownership_None'], axis = 1)

In [30]:
X

,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length_0 year,emp_length_1 year,emp_length_11 years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,term_36 months,term_60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,purpose_Car,purpose_Credit card,purpose_Debt consolidation,purpose_Educational,purpose_Home improvement,purpose_House,purpose_Major purchase,purpose_Medical,purpose_Moving,purpose_Other,purpose_Renewable_energy,purpose_Small business,purpose_Vacation,purpose_Wedding,job_title_Business,job_title_Corporations,job_title_Education,job_title_Facilities,job_title_Financial Services,job_title_Healthcare,job_title_Insurance Services,job_title_Military,job_title_None,job_title_Organizations,job_title_Public Service,job_title_Retail,job_title_Self-Employed,job_title_Service Providers,issue_date_month_April,issue_date_month_August,issue_date_month_December,issue_date_month_February,issue_date_month_January,issue_date_month_July,issue_date_month_June,issue_date_month_March,issue_date_month_May,issue_date_month_November,issue_date_month_October,issue_date_month_September,address_state_full_update_Alabama,address_state_full_update_Alaska,address_state_full_update_Arizona,address_state_full_update_Arkansas,address_state_full_update_California,address_state_full_update_Colorado,address_state_full_update_Connecticut,address_state_full_update_Delaware,address_state_full_update_District of Columbia,address_state_full_update_Florida,address_state_full_update_Georgia,address_state_full_update_Hawaii,address_state_full_update_Idaho,address_state_full_update_Illinois,address_state_full_update_Indiana,address_state_full_update_Iowa,address_state_full_update_Kansas,address_state_full_update_Kentucky,address_state_full_update_Louisiana,address_state_full_update_Maine,address_state_full_update_Maryland,address_state_full_update_Massachusetts,address_state_full_update_Michigan,address_state_full_update_Minnesota,address_state_full_update_Missouri,address_state_full_update_Montana,address_state_full_update_Nebraska,address_state_full_update_Nevada,address_state_full_update_New Hampshire,address_state_full_update_New Jersey,address_state_full_update_New Mexico,address_state_full_update_New York,address_state_full_update_North Carolina,address_state_full_update_Ohio,address_state_full_update_Oregon,address_state_full_update_Others,address_state_full_update_Pennsylvania,address_state_full_update_Rhode Island,address_state_full_update_South Carolina,address_state_full_update_Texas,address_state_full_update_Utah,address_state_full_update_Virginia,address_state_full_update_Washington,address_state_full_update_West Virginia,address_state_full_update_Wisconsin,address_state_full_update_Wyoming,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_Mortgage,home_ownership_Other,home_ownership_Own,home_ownership_Rent
0,-0.616624,-1.849281,-1.277124,0.866753,-1.178995,-1.591672,-1.243439,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-0.336655,-1.196722,-1.039905,1.773553,-1.111977,-1.591672,-0.919717,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Predictions

In [31]:
pred_y = model.predict(X)
 
precision = precision_score(y, pred_y, average='weighted')
recall_ = recall_score(y, pred_y, average='weighted')
f1 = f1_score(y, pred_y, average='weighted')

probs = model.predict_proba(X)[:, 1]
fpr, tpr, thresholds = roc_curve(y, probs)
auc_score = auc(fpr, tpr)

print('Precision:', precision)
print('Recall:', recall_)
print('F1 Score :', f1)
print('AUC Score:', auc_score)
print(classification_report(y, pred_y))

Precision: 0.9912944759235676
Recall: 0.9912639983409374
F1 Score : 0.9911627232608695
AUC Score: 0.996392795260831
              precision    recall  f1-score   support

           0       0.99      0.94      0.97      5333
           1       0.99      1.00      0.99     33243

    accuracy                           0.99     38576
   macro avg       0.99      0.97      0.98     38576
weighted avg       0.99      0.99      0.99     38576



# Create Random dataset

In [146]:
import numpy as np

np.random.seed(20)

num_rows = 1

data = {
    'annual_income': np.random.randint(30000, 100000, num_rows).astype(float),
    'dti': np.random.uniform(0.05, 0.2, num_rows),
    'installment': np.random.uniform(100, 1000, num_rows).astype(float),
    'int_rate': np.random.uniform(0.08, 0.14, num_rows),
    'loan_amount': np.random.randint(5000, 30000, num_rows).astype(float),
    'total_acc': np.random.randint(5, 50, num_rows),
    'total_payment': np.random.randint(1000, 30000, num_rows).astype(float),
    'emp_length': np.random.choice(['0 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10 years', '11 years'], num_rows),
    'term': np.random.choice(['36 months', '60 months'], num_rows),
    'grade': np.random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G'], num_rows),
    'purpose': np.random.choice(['Debt consolidation', 'Credit card', 'Car', 'Home improvement','Small business', 'Major purchase', 'Moving', 'Other', 'House','Vacation', 'Educational', 'Wedding', 'Medical','Renewable_energy'], num_rows),
    'job_title': np.random.choice(['None', 'Organizations', 'Public Service', 'Insurance Services','Service Providers', 'Business', 'Facilities', 'Corporations','Self-Employed', 'Financial Services', 'Education', 'Healthcare','Retail', 'Military'], num_rows),
    'issue_date_month': np.random.choice(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], num_rows),

    'address_state_full_update': np.random.choice(['Georgia', 'California', 'Texas', 'Illinois', 'Pennsylvania','Florida', 'Michigan', 'Rhode Island', 'New York', 'Maryland',
                                                   'Wisconsin', 'Nevada', 'Utah', 'Washington', 'New Hampshire','Hawaii', 'Massachusetts', 'New Jersey', 'Ohio',
                                                   'Arizona', 'Connecticut', 'Minnesota', 'Colorado','Virginia', 'Missouri', 'Delaware', 'New Mexico', 'Louisiana',
                                                   'Arkansas', 'Kentucky', 'North Carolina', 'South Carolina', 'West Virginia', 'Kansas', 'Wyoming', 'Oregon', 'Alabama',
                                                   'District of Columbia', 'Montana', 'Alaska', 'Indiana', 'Maine', 'Idaho', 'Nebraska','Iowa', 'Others'], num_rows),
    
    # 'sub_grade': np.random.choice(['C4', 'E1', 'C5', 'B2', 'A1', 'C3', 'C2', 'A4', 'A5', 'B5', 'B4','B3', 'B1', 'D1', 'A2', 'A3', 'D4', 'D2', 'C1', 
    #                                'D3', 'E3', 'F1','E2', 'E5', 'D5', 'E4', 'F2', 'G3', 'F3', 'G1', 'F4', 'G4', 'G2','F5', 'G5'], num_rows),
    'sub_grade': np.random.choice(['1','2','3','4','5'], num_rows),
    'home_ownership': np.random.choice(['Rent', 'Own', 'Mortgage'], num_rows)
}

df_random = pd.DataFrame(data)
df_random.shape

(1, 16)

In [147]:
df_random['sub_grade']  = df_random['grade'] + df_random['sub_grade'] 

In [148]:
df_random

,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length,term,grade,purpose,job_title,issue_date_month,address_state_full_update,sub_grade,home_ownership
0,67135.0,0.193162,957.050155,0.117289,27294.0,12,20618.0,0 year,36 months,A,Major purchase,Education,November,Arizona,A1,Own


# Input dataset to extract the same features

In [149]:
sql_querys = "SELECT * FROM [Bank Loan].[dbo].[ml_testing_table1]"

In [150]:
df= pd.read_sql_query(sql_querys, connection)
df.shape

(38571, 16)

In [151]:
numerical = ['annual_income', 'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc','total_payment']

In [152]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df[numerical])

In [153]:
def mapping(x):
    if x == '10 > years':
        return '11 years'
    if x == '< 1 year':
        return '0 year'
    else:
        return x
    
df['emp_length'] = df['emp_length'].apply(mapping)

In [154]:
df['job_title'] = df['job_title'].fillna('None')
df.columns

Index(['annual_income', 'dti', 'installment', 'int_rate', 'loan_amount',
       'total_acc', 'total_payment', 'emp_length', 'term', 'grade', 'purpose',
       'job_title', 'issue_date_month', 'address_state_full_update',
       'sub_grade', 'home_ownership'],
      dtype='object')

In [155]:
categorical = ['emp_length', 'term', 'grade', 'purpose', 'job_title', 'issue_date_month', 'address_state_full_update', 'sub_grade','home_ownership']
df_X = pd.get_dummies(df, columns=categorical, dtype=int)
df_X = df_X.drop(columns = ['home_ownership_None'], axis = 1)

# Combine the features with the random input

In [156]:
numerical = ['annual_income', 'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc','total_payment']

In [157]:
testing = pd.get_dummies(df_random, columns=categorical, dtype=int)
testing = testing.reindex(columns=df_X.columns)

testing_numerical = testing[numerical]

testing_scaled = scaler.transform(testing_numerical)

testing[numerical] = testing_scaled

testing

,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length_0 year,emp_length_1 year,emp_length_11 years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,term_36 months,term_60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,purpose_Car,purpose_Credit card,purpose_Debt consolidation,purpose_Educational,purpose_Home improvement,purpose_House,purpose_Major purchase,purpose_Medical,purpose_Moving,purpose_Other,purpose_Renewable_energy,purpose_Small business,purpose_Vacation,purpose_Wedding,job_title_Business,job_title_Corporations,job_title_Education,job_title_Facilities,job_title_Financial Services,job_title_Healthcare,job_title_Insurance Services,job_title_Military,job_title_None,job_title_Organizations,job_title_Public Service,job_title_Retail,job_title_Self-Employed,job_title_Service Providers,issue_date_month_April,issue_date_month_August,issue_date_month_December,issue_date_month_February,issue_date_month_January,issue_date_month_July,issue_date_month_June,issue_date_month_March,issue_date_month_May,issue_date_month_November,issue_date_month_October,issue_date_month_September,address_state_full_update_Alabama,address_state_full_update_Alaska,address_state_full_update_Arizona,address_state_full_update_Arkansas,address_state_full_update_California,address_state_full_update_Colorado,address_state_full_update_Connecticut,address_state_full_update_Delaware,address_state_full_update_District of Columbia,address_state_full_update_Florida,address_state_full_update_Georgia,address_state_full_update_Hawaii,address_state_full_update_Idaho,address_state_full_update_Illinois,address_state_full_update_Indiana,address_state_full_update_Iowa,address_state_full_update_Kansas,address_state_full_update_Kentucky,address_state_full_update_Louisiana,address_state_full_update_Maine,address_state_full_update_Maryland,address_state_full_update_Massachusetts,address_state_full_update_Michigan,address_state_full_update_Minnesota,address_state_full_update_Missouri,address_state_full_update_Montana,address_state_full_update_Nebraska,address_state_full_update_Nevada,address_state_full_update_New Hampshire,address_state_full_update_New Jersey,address_state_full_update_New Mexico,address_state_full_update_New York,address_state_full_update_North Carolina,address_state_full_update_Ohio,address_state_full_update_Oregon,address_state_full_update_Others,address_state_full_update_Pennsylvania,address_state_full_update_Rhode Island,address_state_full_update_South Carolina,address_state_full_update_Texas,address_state_full_update_Utah,address_state_full_update_Virginia,address_state_full_update_Washington,address_state_full_update_West Virginia,address_state_full_update_Wisconsin,address_state_full_update_Wyoming,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_Mortgage,home_ownership_Other,home_ownership_Own,home_ownership_Rent
0,-0.042296,0.89835,3.013796,-0.08603,2.144358,-0.890059,0.923066,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [158]:
pred_y = model.predict(testing)
print(pred_y)

[0]


In [126]:
df_random['predicited_label'] = pred_y

In [54]:
df_random['predicited_words'] = df_random['predicited_label'].apply(lambda x: 'Approved' if x == 1 else 'Not Approved')

In [55]:
df_random.head()

,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length,term,grade,purpose,job_title,issue_date_month,address_state_full_update,sub_grade,home_ownership,predicited_label,predicited_words
0,85808.0,0.129431,856.126601,0.122532,26513.0,40,10082.0,10 years,60 months,D,House,Education,March,Missouri,G5,Own,1,Approved
